In [290]:
import pandas as pd
import numpy as np
import localsolver
import sys
import math

In [291]:
C_C_df = pd.read_csv(r'../Data/C_C.csv')
B_C_df = pd.read_csv(r'../Data/B_C.csv')

In [292]:
C_C_matrix = np.ones((174, 174)) * 1000000000
B_C_matrix = np.ones((141, 174)) * 1000000000

In [293]:
# c_c_matrix
OriginID = C_C_df['OriginID'].values.tolist()
DestinID = C_C_df['Destinatio'].values.tolist()
Length = C_C_df['Total_Leng'].values.tolist()
for i in range(len(OriginID)):
    C_C_matrix[OriginID[i]-1][DestinID[i]-1] = Length[i]

In [294]:
# b_c_matrix
OriginID = B_C_df['OriginID'].values.tolist()
DestinID = B_C_df['Destinatio'].values.tolist()
Length = B_C_df['Total_Leng'].values.tolist()
for i in range(len(OriginID)):
    B_C_matrix[OriginID[i]-1][DestinID[i]-1] = Length[i]

### Clean the isolate road segments
---
#### For C_C OD Matrix
- create a list of customer IDs. [0, number of customers]
- Loop the rows, record rows with all 100000000 records
- remove the recorded rows from customer IDs
- remove rows and columns with all 10000000

In [295]:
Customer_IDs = list(range(C_C_matrix.shape[0]))
records = []
for i in range(C_C_matrix.shape[0]):
    row = C_C_matrix[i]
    ave = row.sum()/(C_C_matrix.shape[0] - 1)
    if ave == 1000000000.0:
        records.append(i)

C_C_matrix = np.delete(C_C_matrix, records, axis=0)
C_C_matrix = np.delete(C_C_matrix, records, axis=1)
B_C_matrix = np.delete(B_C_matrix, records, axis=1)

for i in sorted(records, reverse=True):
    del Customer_IDs[i]

#### For B_C OD Matrix

- create a list of Base ID
- record ID of bases with ave of 100000000
- remove row from od matrix
- remove record from Base ID list

In [296]:
Base_IDs = list(range(B_C_matrix.shape[0]))
records = []
for i in range(B_C_matrix.shape[0]):
    row = B_C_matrix[i]
    ave = row.sum()/B_C_matrix.shape[1]
    if ave > 1000000:
        records.append(i)
B_C_matrix = np.delete(B_C_matrix, records, axis=0)
for i in sorted(records, reverse=True):
    del Base_IDs[i]

### Create new OD matrixes based on demands, if demands == 0, remove the corresponding OD matrixes

In [297]:
demands_data = np.random.randint(0, 2, C_C_matrix.shape[0])
remove = []
for i in range(demands_data.shape[0]):
    if demands_data[i] == 0:
        remove.append(i)
        
C_C_matrix = np.delete(C_C_matrix, remove, axis=0)
C_C_matrix = np.delete(C_C_matrix, remove, axis=1)
B_C_matrix = np.delete(B_C_matrix, remove, axis=1)

for i in sorted(remove, reverse=True):
    del Customer_IDs[i]

In [298]:
demands_data = demands_data.tolist()
for i in sorted(remove, reverse=True):
    del demands_data[i]
nb_customers = C_C_matrix.shape[0]
truck_capacity = 10
dist_matrix_data = C_C_matrix.tolist()
dist_depot_data = B_C_matrix.tolist()[20]

nb_trucks = 40


In [299]:
C_C_matrix.shape[1]

77

In [312]:
str_time_limit = "20"
with localsolver.LocalSolver() as ls:
    model = ls.model
    customers_sequences = [model.list(nb_customers) for _ in range(nb_trucks)]
    
    quantity = [None] * nb_trucks
    for k in range(nb_trucks):
        quantity[k] = [model.int(0, demands_data[i]) for i in range(nb_customers)]
#     print(quantity[0][0].value)
        
    model.constraint(model.cover(customers_sequences))
    
    dist_matrix = model.array(dist_matrix_data)
    dist_depot = model.array(dist_depot_data)
    
    route_distances = [None] * nb_trucks
    trucks_used = [None] * nb_trucks
    
    for k in range(nb_trucks):
        sequence = customers_sequences[k]
        c = model.count(sequence)
        
        trucks_used[k] = model.count(sequence) > 0
        
        quantity_array = model.array(quantity[k])
        quantity_lambda = model.lambda_function(lambda j: quantity_array[j])
        route_quantity = model.sum(sequence, quantity_lambda)
        model.constraint(route_quantity <= truck_capacity)
        
        dist_lambda = model.lambda_function(lambda i: model.at(dist_matrix, sequence[i - 1], sequence[i]))
        route_distances[k] = model.sum(model.range(1, c), dist_lambda) + model.iif(
                    trucks_used[k],
                    dist_depot[sequence[0]] + dist_depot[sequence[c - 1]], 0)
        
    for i in range(nb_customers):
        # Each customer must receive at least its demand
        quantity_served = model.sum(quantity[k][i] * model.contains(customers_sequences[k], i)
            for k in range(nb_trucks))
    
        model.constraint(quantity_served >= demands_data[i])
            
    total_distance = model.sum(route_distances)
    
    model.minimize(total_distance)
#     model.close()
    model.close()

    ls.param.time_limit = int(str_time_limit)
    ls.solve()
    
    r_total_distance = total_distance.value
    customer_list = []
    quantity_list = []
    for k in range(nb_trucks):
        c_list = []
        c
        q_list = []
#         print(trucks_used[k].value)
        if trucks_used[k].value != 1:
            customer_list.append([])
            quantity_list.append([])
            continue

        for customer in customers_sequences[k].value:
            c_list.append(Customer_IDs[customer])

        for c in c_list:
            q_list.append(quantity[k][c].value)
#             if q.value > 0:
#                 q_list.append(q.value)
#             for i in range(len(quantity[k])):
#                 if q[i].value != 0:
#                     q_l.append(q[i].value)
#             q_list.append(q_l)
        customer_list.append(c_list)
        quantity_list.append(q_list)

Push initial solution 100%
Model:  expressions = 13520, decisions = 3120, constraints = 118, objectives = 1
Param:  time limit = 20 sec, no iteration limit

[objective direction ]:     minimize

[  0 sec,       0 itr]: No feasible solution found (infeas = 78)
[  1 sec,    2186 itr]:      40541.5
[  2 sec,   32859 itr]:      30218.5
[  3 sec,   59513 itr]:      30218.5
[  4 sec,   88020 itr]:      30218.5
[  5 sec,  120000 itr]:      30218.5
[  6 sec,  150598 itr]:      30218.5
[  7 sec,  179287 itr]:      29912.9
[  8 sec,  207220 itr]:      29912.9
[  9 sec,  240000 itr]:      29912.9
[ 10 sec,  273858 itr]:      29912.9
[ optimality gap     ]:      100.00%
[ 11 sec,  303253 itr]:      29912.9
[ 12 sec,  332633 itr]:      29912.9
[ 13 sec,  360000 itr]:      29912.9
[ 14 sec,  394996 itr]:      29912.9
[ 15 sec,  422589 itr]:      29912.9
[ 16 sec,  447592 itr]:      29912.9
[ 17 sec,  480000 itr]:      29912.9
[ 18 sec,  508228 itr]:      29912.9
[ 19 sec,  534859 itr]:      29912.9


IndexError: list index out of range

In [309]:
for q in quantity_list:
    if q != []:
        print(sum(q))

10
10
10
10
10
7
10
10


In [310]:
for q in quantity_list:
    print(q)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[]
[1, 1, 1, 1, 1, 1, 1]
[]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[]
[]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
[]
[]
[]


In [311]:
customer_list

[[32, 36, 35, 44, 43, 38, 37, 30, 28, 16],
 [],
 [47, 46, 51, 55, 50, 42, 45, 40, 31, 29],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [47, 48, 49, 60, 64, 70, 76, 75, 74, 73, 56, 29],
 [15, 26, 27, 39, 33, 9, 4, 2, 3, 7],
 [40, 63, 59, 67, 65, 66, 62, 41, 13, 8, 10],
 [],
 [24, 23, 22, 21, 18, 17, 19],
 [],
 [12, 5, 0, 1, 6, 14, 11, 25, 20, 34],
 [],
 [],
 [52, 54, 68, 58, 61, 72, 71, 69, 57, 53, 29],
 [],
 [],
 []]

In [305]:
whole_c = []
for data in customer_list:
    for d in data:
        whole_c.append(d)

In [306]:
len(set(whole_c))


77

In [307]:
demands_data.sum()

AttributeError: 'list' object has no attribute 'sum'

In [82]:
customer_list

[[88, 93, 87, 94, 109, 112, 108],
 [107, 115, 119, 126],
 [32],
 [134, 167, 168, 171, 172, 169, 157, 141],
 [9, 14, 33],
 [92, 150, 138, 146, 130, 74],
 [8, 84, 86, 124],
 [137, 155, 128, 97],
 [85, 166, 165, 164],
 [122, 140, 136, 127, 125, 123, 118, 117, 111, 120],
 [34, 25, 24, 29, 43],
 [35, 21, 22, 18, 28],
 [],
 [85, 74, 58, 52],
 [97, 101, 96, 98, 91],
 [0, 3, 10, 19, 7, 16],
 [],
 [],
 [153, 149, 135, 132, 131, 49],
 [50, 49],
 [27, 5, 13, 26],
 [151, 152, 142, 162, 57],
 [37, 38, 39, 41, 40, 33],
 [91, 170, 163, 159, 158, 154, 49],
 [30, 45, 48, 51],
 [4, 1, 2, 12],
 [100, 99, 106],
 [54, 60, 30],
 [103, 110, 114, 116],
 [59, 77, 83, 80, 65, 62, 47],
 [81, 89, 90, 72, 76, 73, 64],
 [42, 31, 44],
 [66, 71, 78, 53, 55, 56, 57],
 [148, 144, 147, 143, 129],
 [23, 17, 15, 11],
 [6, 20, 46, 36, 67, 68],
 [95, 104, 105, 102],
 [145, 156, 160, 161, 49],
 [79, 69, 61, 63, 70, 82, 75],
 [113, 121, 133, 139]]

In [23]:
print(r_total_distance)

20076814.55788112


In [24]:
customer_list

[[],
 [],
 [],
 [],
 [7, 55, 61, 63, 66, 81, 84, 78, 60, 52, 49],
 [21, 25, 30, 26, 37],
 [57, 67, 72, 79, 54, 56, 50, 44],
 [],
 [114, 106, 102, 97, 99, 107, 123],
 [33, 14, 8, 20, 3, 0, 1, 47],
 [],
 [6, 12, 27, 35, 28, 13, 5, 4, 2],
 [109, 137, 140, 135],
 [110, 88, 94, 89, 95, 86, 59, 53, 51, 43, 32, 45],
 [],
 [],
 [],
 [],
 [105,
  103,
  116,
  142,
  154,
  155,
  159,
  160,
  148,
  145,
  149,
  144,
  147,
  164,
  171,
  163,
  152,
  127],
 [],
 [9, 85, 87, 125, 139, 131, 130, 17],
 [113, 122, 134, 129, 138, 146, 156, 157, 161, 167, 162, 166, 169, 165],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [80, 70, 62, 64, 71, 65, 74, 77, 73, 83, 91, 93, 92, 96, 68],
 [],
 [69, 34, 38, 41, 42, 39, 40],
 [],
 [16, 31, 46, 48, 36, 22, 23, 19, 24, 29, 18, 15, 11, 10],
 [58, 75, 76, 82, 90],
 [],
 [141,
  128,
  126,
  124,
  119,
  118,
  112,
  120,
  121,
  117,
  115,
  111,
  104,
  100,
  101,
  108,
  98,
  96],
 [],
 [132, 133, 136, 150, 151, 143, 153, 158, 170, 173, 172, 168, 113],
 [

In [ ]:
nb_customers = 174
truck_capacity = 5
dist_matrix_data = C_C_matrix.tolist()
dist_depot_data = B_C_matrix.tolist()[20]
demands_data = np.random.randint(0, 2, nb_customers)
nb_trucks = 40
str_time_limit = "50"

In [42]:
dist_depot_data

[1300.246823,
 1082.628237,
 874.5101578,
 1318.158443,
 922.3651787,
 963.7276419,
 870.1246349,
 1045.635534,
 1094.671976,
 909.2017114,
 1196.776467,
 1141.239976,
 648.340294,
 645.0517512,
 741.921893,
 1187.669634,
 853.4033343,
 10000000.0,
 1211.04004,
 1405.664773,
 1262.76636,
 1089.495207,
 1462.740495,
 1406.060137,
 1300.424857,
 798.7429946,
 601.4288034,
 470.857249,
 566.5050269,
 1265.520957,
 706.5121583,
 1276.916631,
 473.9920765,
 382.8078362,
 508.1920004,
 520.6685225,
 1551.428774,
 1405.992641,
 789.8610443,
 938.3642376,
 938.3642376,
 933.0038702,
 933.0038702,
 390.7546435,
 296.7330727,
 345.8450358,
 1417.785646,
 1472.470585,
 1563.06877,
 1652.513195,
 198.1958758,
 245.1326614,
 1798.016979,
 120.3290476,
 46.923712,
 1694.364113,
 16.0086498,
 46.50024105,
 124.4295311,
 353.5631474,
 1719.939694,
 1774.505838,
 549.8603279,
 1767.580794,
 545.3741273,
 624.804996,
 1838.243159,
 44.06487522,
 1251.76477,
 690.0624098,
 449.3068894,
 597.2453804,
 101

In [60]:
for c_list in customer_list:
    distance = 0
    if len(c_list) != 0:
        for i in range(len(c_list) - 1):

            fromC = c_list[i] - 1
            toC = c_list[i+1] - 1

            if i == 0:
                distance += dist_depot_data[fromC]
            else:
                distance += dist_matrix_data[fromC][toC]
        distance += dist_depot_data[c_list[-1] - 1]
    
    print(c_list)
    print(distance)
    print("++++++++++++++++++++++++++++++++++++++++++")

[]
0
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[33, 12, 6, 2, 1, 3, 0, 4, 5, 13, 35]
10913.6420748
++++++++++++++++++++++++++++++++++++++++++
[102, 97, 99, 98, 93, 91, 71, 64, 62, 70, 76]
6858.511611329999
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[123, 134, 135, 168, 169, 172, 173, 170, 158, 153, 142, 116]
7535.460405899999
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[67, 72, 79, 54, 56]
9077.018055
++++++++++++++++++++++++++++++++++++++++++
[57, 92, 117, 130, 139, 147, 144, 149, 145, 148, 160, 17]
6128.1055095
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[]
0
++++++++++++++++++++++++++++++++++++++++++
[96, 88, 94, 89, 95, 105, 122, 140, 141, 165, 137, 128, 126, 124, 119, 118, 112, 120, 121, 116]
15879.97022559
+++++++++++++++++++++++++++++++++++++++

In [57]:
len(dist_depot_data)

174

In [48]:
distance

10405.4500744

In [43]:
customer_list

[[],
 [],
 [33, 12, 6, 2, 1, 3, 0, 4, 5, 13, 35],
 [102, 97, 99, 98, 93, 91, 71, 64, 62, 70, 76],
 [],
 [],
 [123, 134, 135, 168, 169, 172, 173, 170, 158, 153, 142, 116],
 [],
 [67, 72, 79, 54, 56],
 [57, 92, 117, 130, 139, 147, 144, 149, 145, 148, 160, 17],
 [],
 [],
 [96,
  88,
  94,
  89,
  95,
  105,
  122,
  140,
  141,
  165,
  137,
  128,
  126,
  124,
  119,
  118,
  112,
  120,
  121,
  116],
 [103, 107, 108, 101, 100, 104, 111],
 [127, 152, 162, 167, 166, 161, 157, 156, 146, 138, 129],
 [],
 [18, 11, 8, 7, 16],
 [65, 74, 77, 73, 83, 82],
 [57, 86, 80, 75, 59, 58],
 [51, 43, 32, 45, 34, 42, 41, 39, 40, 38, 57],
 [],
 [],
 [57, 50, 44, 28, 27, 26, 30, 25, 21, 47, 37, 68, 69],
 [],
 [],
 [53, 31, 36, 22, 23, 19, 24, 29, 15, 20, 10, 9, 14, 57],
 [115, 132, 133, 136, 150, 154, 155, 159, 164, 171, 163],
 [],
 [96, 110, 109, 113, 114, 106, 90, 82],
 [],
 [],
 [44, 60, 78, 84, 81, 66, 63, 61, 55, 53, 57],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [143, 151, 131, 125, 87, 85, 48, 49, 52, 46